In [1]:
import pandas as pd

In [2]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

/home/roland/Projects/JP_citation_classification/fewshotclassifier/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import Dataset

In [35]:
test = pd.read_csv('/home/roland/Projects/JP_citation_classification/SciBERT_additional_features/data/non_contiguous_acl_arc_exp3/test.txt' ,sep="\t", engine="python", dtype=object)
train = pd.read_csv('/home/roland/Projects/JP_citation_classification/SciBERT_additional_features/data/non_contiguous_acl_arc_exp3/train.txt' ,sep="\t", engine="python", dtype=object)
valid = pd.read_csv('/home/roland/Projects/JP_citation_classification/SciBERT_additional_features/data/non_contiguous_acl_arc_exp3/valid.txt' ,sep="\t", engine="python", dtype=object)

In [24]:
train = train[['citation_class_label', 'dynamic_contexts_combined', 'section_title']]
test = test[['citation_class_label', 'dynamic_contexts_combined', 'section_title']]
valid = valid[['citation_class_label', 'dynamic_contexts_combined', 'section_title']]


In [25]:
train.citation_class_label = train.citation_class_label.astype(int)
test.citation_class_label = test.citation_class_label.astype(int)
valid.citation_class_label = valid.citation_class_label.astype(int)

In [26]:
train = train.loc[(train.citation_class_label == 1) | (train.citation_class_label == 0)]
test = test.loc[(test.citation_class_label == 1) | (test.citation_class_label == 0)]
valid = valid.loc[(valid.citation_class_label == 1) | (valid.citation_class_label == 0)]

In [27]:
train = train[['dynamic_contexts_combined', 'citation_class_label', 'section_title']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})
test = test[['dynamic_contexts_combined', 'citation_class_label', 'section_title']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})
valid = valid[['dynamic_contexts_combined', 'citation_class_label', 'section_title']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})

In [37]:
train = train[['citation_class_label', 'dynamic_contexts_combined']]
test = test[['citation_class_label', 'dynamic_contexts_combined']]
valid = valid[['citation_class_label', 'dynamic_contexts_combined']]
train.citation_class_label = train.citation_class_label.astype(int)
test.citation_class_label = test.citation_class_label.astype(int)
valid.citation_class_label = valid.citation_class_label.astype(int)
train = train.loc[(train.citation_class_label == 1) | (train.citation_class_label == 0)]
test = test.loc[(test.citation_class_label == 1) | (test.citation_class_label == 0)]
valid = valid.loc[(valid.citation_class_label == 1) | (valid.citation_class_label == 0)]
train = train[['dynamic_contexts_combined', 'citation_class_label']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})
test = test[['dynamic_contexts_combined', 'citation_class_label']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})
valid = valid[['dynamic_contexts_combined', 'citation_class_label']].rename(columns={'dynamic_contexts_combined': 'text', 'citation_class_label': 'label'})

In [38]:
train_ds_full = Dataset.from_pandas(train).remove_columns('__index_level_0__')
test_ds = Dataset.from_pandas(test).remove_columns('__index_level_0__')

In [39]:
train_ds = train_ds_full.shuffle(seed=34).select(range(8 * 2))
#test_ds = dataset["test"]

In [40]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [41]:
trainer.train()
metrics = trainer.evaluate()

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 1657.14it/s]
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [00:15<00:00, 15.68s/it]
***** Running evaluation *****


In [33]:
metrics

{'accuracy': 0.7268041237113402}

In [42]:
metrics

{'accuracy': 0.7268041237113402}

In [43]:
trainer.model._save_pretrained(save_directory="./output/")